In [ ]:
# 定制类
# 看到类似__slots__这种形如__xxx__的变量或者函数名就要注意，这些在Python中是有特殊用途的。
# __slots__我们已经知道怎么用了，__len__()方法我们也知道是为了能让class作用于len()函数。
# 除此之外，Python的class中还有许多这样有特殊用途的函数，可以帮助我们定制类。
# __str__
# 我们先定义一个Student类，打印一个实例：
class Student():
    def __init__(self,name):
        self.name = name
    
print(Student('Quwenxi'))

In [2]:
# 打印出一堆<__main__.Student object at 0x109afb190>，不好看。
# 怎么才能打印得好看呢？只需要定义好__str__()方法，返回一个好看的字符串就可以了：
class Student(object):
     def __init__(self, name):
         self.name = name
     def __str__(self):
         return 'Student object (name: %s)' % self.name
     
print(Student('Quwenxi'))

Student object (name: Quwenxi)


In [ ]:
# 但是，给一个实例绑定的方法，对另一个实例是不起作用的：
s2 = Student() # 创建新的实例
s2.set_age(25) # 尝试调用方法

AttributeError: 'Student' object has no attribute 'set_age'

In [4]:
# 这样打印出来的实例，不但好看，而且容易看出实例内部重要的数据。
# 但是细心的朋友会发现直接敲变量不用print，打印出来的实例还是不好看：
s = Student('Michael')
s

In [5]:
# 这是因为直接显示变量调用的不是__str__()，而是__repr__()，
# 两者的区别是__str__()返回用户看到的字符串，而__repr__()返回程序开发者看到的字符串，也就是说，__repr__()是为调试服务的。
# 解决办法是再定义一个__repr__()。但是通常__str__()和__repr__()代码都是一样的，所以，有个偷懒的写法：
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name=%s)' % self.name
    __repr__ = __str__

s = Student('Michael')
s

Student object (name=Michael)

In [6]:
# __iter__
# 如果一个类想被用于for ... in循环，类似list或tuple那样，就必须实现一个__iter__()方法，
# 该方法返回一个迭代对象，然后，Python的for循环就会不断调用该迭代对象的__next__()方法拿到循环的下一个值，直到遇到StopIteration错误时退出循环。
# 我们以斐波那契数列为例，写一个Fib类，可以作用于for循环：
class Fib():
    def __init__(self):
        self.a,self.b = 0,1

    def __iter__(self):
        return self
    
    def __next__(self):
        self.a,self.b = self.b,self.a+self.b
        if self.a>100000:
            raise StopIteration
        return self.a
    
# 现在，试试把Fib实例作用于for循环：
for n in Fib():
    print(n)

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025


In [7]:
# __getitem__
# Fib实例虽然能作用于for循环，看起来和list有点像，但是，把它当成list来使用还是不行，比如，取第5个元素：
Fib()[5]

TypeError: 'Fib' object is not subscriptable

In [ ]:
# 要表现得像list那样按照下标取出元素，需要实现__getitem__()方法：
class Fib(object):
    def __getitem__(self, n):
        a, b = 1, 1
        for x in range(n):
            a, b = b, a + b
        return a

# 现在，就可以按下标访问数列的任意一项了：
f = Fib()
print(f[0])

print(f[1])

print(f[2])

print(f[3])

print(f[10])

print(f[100])

1
1
2
3
89
573147844013817084101


In [10]:
# 但是list有个神奇的切片方法：
list(range(100))[5:10]

[5, 6, 7, 8, 9]

In [11]:
# 对于Fib却报错。原因是__getitem__()传入的参数可能是一个int，也可能是一个切片对象slice，所以要做判断：
class Fib(object):
    def __getitem__(self, n):
        if isinstance(n, int): # n是索引
            a, b = 1, 1
            for x in range(n):
                a, b = b, a + b
            return a
        if isinstance(n, slice): # n是切片
            start = n.start
            stop = n.stop
            if start is None:
                start = 0
            a, b = 1, 1
            L = []
            for x in range(stop):
                if x >= start:
                    L.append(a)
                a, b = b, a + b
            return L

In [12]:
f = Fib()
f[0:5]

[1, 1, 2, 3, 5]

In [13]:
f[:10]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [ ]:
# 但是没有对step参数作处理：
f[:10:2]
# 也没有对负数作处理，所以，要正确实现一个__getitem__()还是有很多工作要做的。
# 此外，如果把对象看成dict，__getitem__()的参数也可能是一个可以作key的object，例如str。
# 与之对应的是__setitem__()方法，把对象视作list或dict来对集合赋值。最后，还有一个__delitem__()方法，用于删除某个元素。
# 总之，通过上面的方法，我们自己定义的类表现得和Python自带的list、tuple、dict没什么区别，这完全归功于动态语言的“鸭子类型”，不需要强制继承某个接口。

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [15]:
# __getattr__
# 正常情况下，当我们调用类的方法或属性时，如果不存在，就会报错。比如定义Student类：
class Student(object):
    def __init__(self):
        self.name = 'Michael'

In [ ]:
# 调用name属性，没问题，但是，调用不存在的score属性，就有问题了：
s = Student()
print(s.name)
print(s.score)
# 错误信息很清楚地告诉我们，没有找到score这个attribute。

Michael


AttributeError: 'Student' object has no attribute 'score'

In [19]:
# 要避免这个错误，除了可以加上一个score属性外，Python还有另一个机制，那就是写一个__getattr__()方法，动态返回一个属性。修改如下：
class Student(object):
    def __init__(self):
        self.name = 'Michael'

    def __getattr__(self, attr):
        return 99

# 当调用不存在的属性时，比如score，Python解释器会试图调用__getattr__(self, 'score')来尝试获得属性，这样，我们就有机会返回score的值：
s = Student()
print(s.name)
print(s.score)
print(s.play)


Michael
99
99


In [20]:
# 返回函数也是完全可以的：
class Student(object):
    def __getattr__(self, attr):
        if attr=='age':
            return lambda: 25
        
s = Student()
print(s.age())

25


In [21]:
# 注意，只有在没有找到属性的情况下，才调用__getattr__，已有的属性，比如name，不会在__getattr__中查找。
# 此外，注意到任意调用如s.abc都会返回None，这是因为我们定义的__getattr__默认返回就是None。
# 要让class只响应特定的几个属性，我们就要按照约定，抛出AttributeError的错误：
class Student(object):
    def __getattr__(self, attr):
        if attr=='age':
            return lambda: 25
        raise AttributeError('\'Student\' object has no attribute \'%s\'' % attr)

In [ ]:
# 这实际上可以把一个类的所有属性和方法调用全部动态化处理了，不需要任何特殊手段。

# 这种完全动态调用的特性有什么实际作用呢？作用就是，可以针对完全动态的情况作调用。

# 举个例子：

# 现在很多网站都搞REST API，比如新浪微博、豆瓣啥的，调用API的URL类似：

# http://api.server/user/friends
# http://api.server/user/timeline/list
# 如果要写SDK，给每个URL对应的API都写一个方法，那得累死，而且，API一旦改动，SDK也要改。

# 利用完全动态的__getattr__，我们可以写出一个链式调用：
class Chain(object):
    def __init__(self, path=''):
        self._path = path

    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))

    def __str__(self):
        return self._path

    __repr__ = __str__

Chain().status.user.timeline.list

# 这样，无论API怎么变，SDK都可以根据URL实现完全动态的调用，而且，不随API的增加而改变！
# 还有些REST API会把参数放到URL中，比如GitHub的API：

/status/user/timeline/list

In [23]:

# 还有些REST API会把参数放到URL中，比如GitHub的API：
# GET /users/:user/repos
# 调用时，需要把:user替换为实际用户名。如果我们能写出这样的链式调用：
Chain().users('michael').repos

TypeError: 'Chain' object is not callable

In [ ]:
class Chain(object):
    def __init__(self, path=''):
        self._path = path

    def __call__(self, *args, **kwargs):
        if args:
            return Chain('%s/%s' % (self._path, '/'.join(map(str, args))))
        elif kwargs:
            return Chain('%s/%s' % (self._path, '/'.join(kwargs.keys())))
        else:
            raise TypeError("Chain() missing 1 required positional argument")

    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))

    def __str__(self):
        return self._path

    __repr__ = __str__

# 使用示例
chain = Chain()
print(chain.users('michael').repos)  # 输出: users/michael/repos

# 当尝试访问一个不存在的属性时，Python 会调用 __getattr__ 方法。
# 在这个方法中，path 是被访问的属性名。
# 每次调用时，它会创建一个新的 Chain 实例，并将当前路径与新路径拼接，用 / 分隔。
# 例如，chain.users 会触发 __getattr__，将 self._path（初始为空）与 'users' 拼接，返回一个新的 Chain 实例，其路径为 'users'。

# 当实例被当作函数调用时（例如 chain.users('michael')），Python 会调用 __call__ 方法。
# 这个方法支持两种调用方式：
# 位置参数（args）：将所有位置参数转换为字符串并用 / 拼接，然后与当前路径拼接。
# 关键字参数（kwargs）：将关键字参数的键（keys()）用 / 拼接，然后与当前路径拼接。
# 如果没有提供任何参数，则抛出 TypeError。

# __str__ 方法定义了对象的字符串表示形式，返回当前路径 self._path。
# __repr__ 方法通常用于调试，这里将其设置为与 __str__ 相同，确保在交互式环境中打印对象时也能显示路径。

/users/michael/repos


In [1]:
# __call__
# 一个对象实例可以有自己的属性和方法，
# 当我们调用实例方法时，我们用instance.method()来调用。能不能直接在实例本身上调用呢？在Python中，答案是肯定的。
# 任何类，只需要定义一个__call__()方法，就可以直接对实例进行调用。请看示例：
class Student(object):
    def __init__(self, name):
        self.name = name

    def __call__(self):
        print('My name is %s.' % self.name)

s = Student('Michel')
s()

My name is Michel.


In [ ]:
# __call__()还可以定义参数。
# 对实例进行直接调用就好比对一个函数进行调用一样，所以你完全可以把对象看成函数，
# 把函数看成对象，因为这两者之间本来就没啥根本的区别。

# 如果你把对象看成函数，那么函数本身其实也可以在运行期动态创建出来，
# 因为类的实例都是运行期创建出来的，这么一来，我们就模糊了对象和函数的界限。

# 那么，怎么判断一个变量是对象还是函数呢？其实，更多的时候，我们需要判断一个对象是否能被调用，
# 能被调用的对象就是一个Callable对象，比如函数和我们上面定义的带有__call__()的类实例：

print(callable(Student('Michel')))

print(callable(max))

print(callable([1, 2, 3]))

print(callable(None))

print(callable('str'))

# 通过callable()函数，我们就可以判断一个对象是否是“可调用”对象。

True
True
False
False
False


In [ ]:
# 小结
# Python的class允许定义许多定制方法，可以让我们非常方便地生成特定的类。
# 本节介绍的是最常用的几个定制方法，还有很多可定制的方法，请参考Python的官方文档。